In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import indexer.database as db

In [2]:
from bokeh.io import output_notebook, show, save

import networkx as nx
import pandas as pd
import numpy as np

In [28]:
output_notebook()

Loading BokehJS ...

In [29]:
with open('private/postgres_password', 'r') as f:
    password = f.read()
    
with open('private/postgres_host', 'r') as f:
    host = f.read()
    
engine = create_engine(f'postgresql://postgres:{password}@{host}:5432/ton_index')

# Getting transactions

In [13]:
from indexer.database import Block, BlockHeader, and_
from dataclasses import asdict, is_dataclass
from tqdm.auto import tqdm

In [14]:
def unpack_msg(msg):
    if is_dataclass(msg):
        res = asdict(msg)
        res.pop('msg_id', None)
        content = asdict(msg.content)
        content.pop('msg_id', None)
        res.update(content)
        return res
    return None

In [17]:
seqno = 19938504

fltr = and_(Block.seqno == seqno, Block.workchain == -1)

tx_list = []
Session = sessionmaker(bind=engine)
with Session() as session:
    master = session.query(Block).filter(fltr).first()
    blocks = [master] + master.shards
    
    txs = []
    for b in blocks:
        txs.extend(b.transactions)

    for tx in txs:
        t = asdict(tx)
        t.pop('tx_id')
        # decode in tx
        t['in_msg'] = [unpack_msg(x) for x in tx.in_msg]
        t['out_msgs'] = [unpack_msg(x) for x in tx.out_msgs]
        tx_list.append(t)

In [18]:
tx_list

[{'account': '-1:3333333333333333333333333333333333333333333333333333333333333333',
  'lt': 27238580000001,
  'hash': '/Inp21HZBRd7pOl4rkuHeSx8iReC+llzbONlQ/Buh9s=',
  'utime': 1650395570,
  'fee': 0,
  'storage_fee': 0,
  'other_fee': 0,
  'in_msg': [],
  'out_msgs': []},
 {'account': '-1:3333333333333333333333333333333333333333333333333333333333333333',
  'lt': 27238580000002,
  'hash': 'mzCIURN5BLMYvFOL1UCjjI4Of4eDLhm6vJ1mjSGT5kg=',
  'utime': 1650395570,
  'fee': 0,
  'storage_fee': 0,
  'other_fee': 0,
  'in_msg': [{'source': 'Ef8AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAADAU',
    'destination': 'Ef8zMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzM0vF',
    'value': 3714802973,
    'fwd_fee': 0,
    'ihr_fee': 0,
    'created_lt': 27238580000000,
    'body_hash': 'lqKW0iTyhcZ77pPDD4owkVfw2qNdxbh+QQt4YwoJz8c=',
    'body': 'te6cckEBAQEAAgAAAEysuc0='}],
  'out_msgs': []},
 {'account': '-1:34517c7bdf5187c55af4f8b61fdc321588c7ab768dee24b006df29106458d7cf',
  'lt': 27238580000001,
  'hash': '

# Graph viz

In [ ]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx

from functools import partial

In [ ]:
from indexer.database import Block, BlockHeader, and_
from tqdm.auto import tqdm


start_seqno = 19938494
end_seqno = start_seqno + 64

block_headers = {}
edges = []

fltr = and_(Block.seqno >= start_seqno, Block.seqno < end_seqno, Block.workchain == -1)
prev = None

with Session() as session:
    blocks = session.query(Block).filter(fltr)
    for block in tqdm(blocks):
        block_headers[block.block_id] = (block, block.block_header)
        for shard in block.shards:
            block_headers[shard.block_id] = (shard, shard.block_header)
            edges.append((block.block_id, shard.block_id))
        if prev is not None:
            edges.append((block.block_id, prev.block_id))
        prev = block

# Graph

In [ ]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx

from functools import partial

In [ ]:
G = nx.from_edgelist(edges)
nx.set_node_attributes(G, name='workchain', values={k: v[0].workchain for k, v in block_headers.items()})
nx.set_node_attributes(G, name='shard', values={k: v[0].shard for k, v in block_headers.items()})
nx.set_node_attributes(G, name='seqno', values={k: v[0].seqno for k, v in block_headers.items()})
nx.set_node_attributes(G, name='start_lt', values={k: v[1].start_lt for k, v in block_headers.items()})

In [ ]:
positions = {k: [v[0].workchain, v[1].start_lt / 1000000] for k, v in block_headers.items()}

In [ ]:
# Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [("block_id", "@index"),
                  ("workchain", "@workchain"),
                  ("shard", "@shard"),
                  ("seqno", "@seqno"),
                  ("start_lt", "@start_lt")]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
              x_range=Range1d(-2., 10.),
              # y_range=Range1d(-0.1, 0.1)
             )

#Create a network graph object with spring layout
network_graph = from_networkx(G, positions)

#Set node size and color
network_graph.node_renderer.glyph = Circle(size=10, fill_color='skyblue')

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

#Add network graph to the plot
plot.renderers.append(network_graph)

show(plot)
#save(plot, filename=f"{title}.html")

# TonClient

In [20]:
import json
import os
import asyncio

from tApi.tonlib import TonlibClient


with open('toncenter-game.conf', 'r') as f:
    config = json.load(f)
    
    
keystore = 'private/ton_keystore'
os.makedirs(keystore, exist_ok=True)
loop = asyncio.get_running_loop()

client = TonlibClient(0, config, keystore, loop)
await client.init()

[ 4][t 1][2022-04-20 10:06:41.593678400][TonlibClient.cpp:1479][!Tonlib][&tonlib_query]	Tonlib got query [id:1] setLogVerbosityLevel {
  new_verbosity_level = 0
}
[ 4][t 1][2022-04-20 10:06:41.593730300][TonlibClient.cpp:1518][!Tonlib][&tonlib_query]	Tonlib got static query setLogVerbosityLevel {
  new_verbosity_level = 0
}
[ 4][t 0][2022-04-20 10:06:41.593899100][Client.cpp:78][&tonlib_requests]	Begin to wait for updates with timeout 3.000000
2022-04-20 10:06:41.602 | INFO     | tApi.tonlib.client:init:95 - TonLib #000 inited successfully


In [21]:
await client.lookupBlock(-1, -9223372036854775808, 19938494)
await client.getShards(19938497)

{'@type': 'blocks.shards',
 'shards': [{'@type': 'ton.blockIdExt',
   'workchain': 0,
   'shard': '-9223372036854775808',
   'seqno': 25083806,
   'root_hash': '7sc/lSqb4u5aeF+HkKE9VfBbwBgq/miIzH3wgcFUzz4=',
   'file_hash': 'd4zRyuiWnc164VYOAzEOWT8Y9vzpAlxXN+0XJd/khjA='}],
 '@extra': '1650449212.6697292:0:0.2304340131452618'}

In [22]:
await client.getShards(19938496)

{'@type': 'blocks.shards',
 'shards': [{'@type': 'ton.blockIdExt',
   'workchain': 0,
   'shard': '-9223372036854775808',
   'seqno': 25083804,
   'root_hash': '16tTvNHZ7IWYWpxy1wfqH4MFSZ+qdyuLfzT0oIKfidw=',
   'file_hash': 'vhNgsWRV9ZZScTGWUptB3oybHk4wptd3jfjdnLjdKwo='}],
 '@extra': '1650449213.3943841:0:0.44021540381245317'}

In [25]:
await client.getBlockTransactions(-1, -9223372036854775808, 19938504, 100)

{'@type': 'blocks.transactions',
 'id': {'@type': 'ton.blockIdExt',
  'workchain': -1,
  'shard': '-9223372036854775808',
  'seqno': 19938504,
  'root_hash': 'jFRwthvGWRbV1t6bWZowiB03dqqQXAi+hTvxbpD3sxY=',
  'file_hash': 'UsBSt87ptDDz20gETG6MfWS+Wck3vER1hyHZKOZWOxs='},
 'req_count': 100,
 'incomplete': False,
 'transactions': [{'@type': 'blocks.shortTxId',
   'mode': 135,
   'account': '-1:3333333333333333333333333333333333333333333333333333333333333333',
   'lt': '27238580000001',
   'hash': '/Inp21HZBRd7pOl4rkuHeSx8iReC+llzbONlQ/Buh9s='},
  {'@type': 'blocks.shortTxId',
   'mode': 135,
   'account': '-1:3333333333333333333333333333333333333333333333333333333333333333',
   'lt': '27238580000002',
   'hash': 'mzCIURN5BLMYvFOL1UCjjI4Of4eDLhm6vJ1mjSGT5kg='},
  {'@type': 'blocks.shortTxId',
   'mode': 135,
   'account': '-1:34517c7bdf5187c55af4f8b61fdc321588c7ab768dee24b006df29106458d7cf',
   'lt': '27238580000001',
   'hash': 'onDt/lTymfKdbBfUg/Gr857RTpoqqt74KLaEVNKqRzk='},
  {'@type': '